## 1. 라이브러리 가져오기

In [ ]:
import requests # 웹페이지의 HTML 내용을 가져오는데 사용되는 라이브러리

# bs4 : 파이썬에서 웹 스크래핑 및 데이터 추출을 위한 라이브러리
from bs4 import BeautifulSoup # requests로 가져온 html을 파싱하고, 필요한 데이터를 추출하는 데 사용되는 라이브러리

# 정규표현식을 이용한 문자열 검색/치환/매칭 기능을 제공하는 모듈
import re

# html 특수문자 인코딩/디코딩, html 파싱/엔티티 정보 제공 등을 해주는 모듈
import html

import json

import urllib.parse

## 2. 사용자로부터 검색 키워드를 받아오고, 해당 페이지에서 뉴스 링크 추출

In [ ]:
# << 2. 웹사이트를 방문하고 Parsing >>

# requests의 http 메서드의 반환값인 Response 객체를 받아옴
url = requests.get("https://news.naver.com/section/105")
url.text


# parsing된 결과 확인
    # <BeautifulSoup로 웹페이지의 html을 parsing해서 분석 가능한 구조로 만드는 작업>
html = BeautifulSoup(url.text, 'html.parser')
    # BeautifulSoup 라이브러리를 통해, Response에 담겨있는 html 코드를 객체처럼 다룰 수 있도록 함
        # -> url.text: request.get(url)을 통해 받아온 HTML 텍스트 전체
        # -> html.parser : HTML을 파싱하기 위한 parser 종류 지정 (파이썬 기본 내장)
        # -> BeautifulSoup : HTML 문자열을 "구문 분석(파싱)"해서 쉽게 검색하고 추출할 수 있게 바꿔주는 라이브러리/클래스

# 이제, BeautifulSoup 객체로 html을 객체처럼 다룰 수 있음
html.title

<title>IT/과학 : 네이버 뉴스</title>

In [ ]:
# << 3. 기사 제목과 내용 가져오기 >> #
# 각 기사별 제목, url을 가져와서, 해당 url에 재접속하여 각 기사의 제목, 본문을 가져온다.

    # parsing한 웹사이트에서 각 기사 제목에 있는 기사 링크를 추출함
articles_link = html.select('a.sa_text_title') # <a class="sa_text_title _NLOG_IMPRESSION' ...>

# 각 기사별 데이터를 저장할 배열 생성
articles_data = []

    # 가져온 각 기사 링크에 접속해서, 기사별 제목, 본문을 가져와 articles_data list에 저장한다.
for article in articles_link:
    title = article.get_text(strip=True) # strip=True로 설정하면, 앞 뒤의 공백 등의 개행문자들을 제거해줌
    link = article['href'] # 해당 기사 링크 가져옴

    # 각 기사 페이지 재요청
    article_res = requests.get(link) # Response 가져옴
    article_html = BeautifulSoup(article_res.text, 'html.parser') # Response의 html과, parsing 형식을 지정해주어 BeautifulSoup 객체 가져옴

 #   print(article_html)

    # 해당 기사 페이지의 기사 제목, 본문을 추출한다.
    try:
        # 기사 제목, 본문 가져옴
        article_title = article_html.select_one("h2#title_area span").get_text(strip=True)
        article_body = article_html.select_one("div#newsct_article").get_text(strip=True)
    except:
        article_title = "[제목 없음]"
        article_body = "[본문 없음]"

        # 저장 되었는지 확인용 코드
    # print(f"\n 제목: {article_title}\n")
    # print(f"본문: {article_body[:400]}...") # 본문 앞 400자만 출력

    # 가져온 데이터들을 articles_data에 딕셔너리로 저장한다
    # 기사 제목, 본문, 링크 저장
    articles_data.append({
        "title" : article_title,
        "body" : article_body,
        "url" : link
    })



In [ ]:
# << 4. 데이터 전처리 진행 >> #

    # 4-1) 정제 (Cleaning)

def clean_text(text):
    # 1) HTML 엔티티(특수문자) 디코딩
    t = html.unescape(text)

    # 2) 스마트따옴표·전각문자 통일
    t = t.replace('“', '"').replace('”', '"')

    # 3) 이메일·URL·숫자 플레이스홀더
    # re.X 플래그를 지정해줌으로써, 패턴 내 공백, 주석 허용

    t = re.sub(r'''
        \b  # 단어 경계 (앵커)
        [\w\.-]+ # (문자들(A-Za-z0-9), ".", "-") 중 하나가 1회 이상 반복됨
        @[\w\.-]+ # 위와 동일
        \.\w+ # "." 다음에 단어가 1개 이상 반복됨
        \b  # 단어 경계 종료
    ''', '<EMAIL>', t, re.X) # 해당 형태의 단어 "<EMAIL>" 태그로 변경


    t = re.sub(r'https?://\S+|www\.\S+', '<URL>', t)
    t = re.sub(r'\d+([.,]\d+)*', '<NUM>', t)
    # 4) 불필요 기호 제거
    t = re.sub(r'[※■▶★♡♥]', '', t)
    # 5) 괄호 속 설명 제거
    t = re.sub(r'\[[^\]]*\]|\([^)]*\)', '', t)
    # 6) 제어문자 제거
    t = re.sub(r'[\t\r]', ' ', t)
    # 7) 연속 공백/개행 통일
    t = re.sub(r'\n{3,}', '\n\n', t)
    t = re.sub(r' {2,}', ' ', t)
    # 8) strip
    return t.strip()

# 기사 내용 정제
# articles_data는 딕셔너리의 list이므로, 한번에 한 딕셔너리씩 정제시킨다.
cleaned_articles = []
for article in articles_data:
    new_article = dict(article)  # 원본 복사
    new_article["title"] = clean_text(article["title"])  # title, body만 정제
    new_article["body"] = clean_text(article["body"])  # title, body만 정제
    cleaned_articles.append(new_article)


print(json.dumps(cleaned_articles, indent=2, ensure_ascii=False))